# Data D2


### Data format
The data format is a .pkl file (pickle) containing a dataframe with at least these columns:
- id, a_text, b_id, b_text, c_id, c_text, research_type, y_true

In [5]:
import pandas as pd

all_papers = pd.read_csv("./data/temp/papers_2025.csv")
print(all_papers.shape, "->", all_papers.columns)

(58444, 6) -> Index(['id', 'title', 'abstract', 'categories', 'authors', 'date'], dtype='object')


In [6]:
papersB = all_papers.sample(1000, random_state=42).reset_index(drop=True)

print(papersB.shape, "->", papersB.columns)

(1000, 6) -> Index(['id', 'title', 'abstract', 'categories', 'authors', 'date'], dtype='object')


In [7]:
# Getting a single string for "paper B" and a single string for "paper C"

def concatTitleAbstract(obj):
    if obj.get('abstract', None):
        return f"{obj['title']}; {obj['abstract']}"
    return obj['title']

data = papersB.copy()

data["id"] = None
data["a_text"] = None
data["b_id"] = None
data["b_text"] = None
data["b_categories"] = None
data["c_id"] = None
data["c_text"] = None
data["c_categories"] = None

for i, row in data.iterrows():
    paper_c = papersB.loc[~(papersB.categories == row.categories)].sample(1, random_state=i).reset_index(drop=True).loc[0]
    
    a_text = ""
    b_text = concatTitleAbstract(papersB.loc[i])
    c_text = concatTitleAbstract(paper_c)
    
    data.loc[i, "id"] = f"neg-d2-{i}"
    data.loc[i, "a_text"] = ""
    
    data.loc[i, "b_id"] = papersB.loc[i].id
    data.loc[i, "b_text"] = b_text
    data.loc[i, "b_categories"] = papersB.loc[i].categories
    
    data.loc[i, "c_id"] = paper_c.id
    data.loc[i, "c_text"] = c_text
    data.loc[i, "c_categories"] = paper_c.categories

data["y_true"] = False
data["research_type"] = None
data = data.drop(columns=["title", "abstract", "categories", "authors"])

print(data.shape, "->", data.columns)

(1000, 11) -> Index(['id', 'date', 'a_text', 'b_id', 'b_text', 'b_categories', 'c_id',
       'c_text', 'c_categories', 'y_true', 'research_type'],
      dtype='object')


In [8]:
data.to_pickle("./data/data_D2.pkl")
data = pd.read_pickle("./data/data_D2.pkl")

data

,id,date,a_text,b_id,b_text,b_categories,c_id,c_text,c_categories,y_true,research_type
0,neg-d2-0,2025-03-23,,2503.18058,On the Classification of Isoparametric Hypersu...,['math.DG'],2501.01753,Detecting Turbulent Patterns in Particulate Pi...,['physics.flu-dyn'],False,None
1,neg-d2-1,2025-01-14,,2501.08479,Skyrise: Exploiting Serverless Cloud Infrastru...,['cs.DB'],2502.09337,Some aspects of descent theory and application...,['math.CT'],False,None
2,neg-d2-2,2025-01-29,,2501.17489,Neural Spelling: A Spell-Based BCI System for ...,"['cs.HC', 'cs.AI']",2502.1968,M-LLM Based Video Frame Selection for Efficien...,"['cs.CV', 'cs.AI']",False,None
3,neg-d2-3,2025-01-24,,2501.14901,Oscillatory Line-Driven Winds: The Role of Atm...,['astro-ph.GA'],2502.18295,Quantization of the Momentum Map via $\frak{g}...,"['math.QA', 'math-ph', 'math.MP', 'math.SG']",False,None
4,neg-d2-4,2025-03-06,,2503.04612,On the distribution of the angle between Osele...,['math.DS'],2501.0317,Universality in the microwave shielding of ult...,"['cond-mat.quant-gas', 'physics.atom-ph']",False,None
...,...,...,...,...,...,...,...,...,...,...,...
995,neg-d2-995,2025-01-14,,2501.08056,Magnetic Dichroism in Rutile NiF$_2$: Separati...,"['cond-mat.mtrl-sci', 'cond-mat.str-el']",2501.13297,RAMQA: A Unified Framework for Retrieval-Augme...,"['cs.CL', 'cs.AI', 'cs.IR', 'cs.LG']",False,None
996,neg-d2-996,2025-03-08,,2503.06304,Optimization and Benchmarking of Monolithicall...,['cs.ET'],2503.09695,Modified large-$N$ approach to gapless spin li...,['cond-mat.str-el'],False,None
997,neg-d2-997,2025-03-18,,2503.13993,On a Diophantine Inequality with Primes Yieldi...,['math.NT'],2503.08007,MoRE: Unlocking Scalability in Reinforcement L...,"['cs.RO', 'cs.AI']",False,None
998,neg-d2-998,2025-01-16,,2501.09741,Asymptotic behavior of clusters in hierarchica...,['math.PR'],2502.01312,CleanPose: Category-Level Object Pose Estimati...,['cs.CV'],False,None
